In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import json
from pandas.io.json import json_normalize

import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image

from typing import List, Dict

from sklearn.model_selection import train_test_split

# Preprocess Data

In [3]:
data_location = ['data','/media/danil-pass123/Новый том/DataSet'][1]

In [4]:
os.chdir(data_location)
!ls

archives			     processed_train.csv
gps_locations.json		     test
iwildcam2022_mdv4_detections.json    train
iwildcam2022_test_information.json   train_sequence_counts.csv
iwildcam2022_train_annotations.json


In [4]:
def get_label(filename: str, labels: List[Dict]):
    row = next((x for x in labels if filename == x['file']), [None])
    if row is None:
        return row
    detections = row['detections']
    filtered_detections = list(filter(lambda x: x['category'] == '1' and x['conf'] > 0.5, detections))
    bboxes = list(map(lambda x: x['bbox'], filtered_detections))
    return json.dumps(bboxes)

for every image:
category=='animal' and conf>0.5
return boxes

In [ ]:
def get_data(images_json_path: str, labels_json_path: str):
    d = None
    with open(labels_json_path) as f:
        d = json.load(f)
    labels = d['images']

    with open(images_json_path) as f:
        d = json.load(f)
    images = json_normalize(data=d['images'],
                   meta=['seq_num_frames', 'location', 'datetime', 'id', 'seq_id', 'width', 'height', 'file_name', 'sub_location', 'seq_frame_num'])
    # annotations_df = json_normalize(data=d['annotations'],
    #                              meta=['id', 'image_id', 'category_id'])
    # categories_df = json_normalize(data=d['categories'],
    #                             meta=['id', 'name'])

    # return get_label('train/' + images.iloc[0]['file_name'], labels)
    images['box'] = images.apply(lambda x: get_label('train/' + x['file_name'], labels), axis=1)
    return images



In [ ]:
df = get_data('iwildcam2022_train_annotations.json', 'iwildcam2022_mdv4_detections.json')

In [ ]:
df

In [5]:
df.to_csv('processed_train.csv', sep='\t')

NameError: name 'df' is not defined

In [56]:
df = pd.read_csv('processed_train.csv', sep='\t', index_col=0)
df.head()

,seq_num_frames,location,datetime,id,seq_id,width,height,file_name,sub_location,seq_frame_num,box
0,6,3,2013-06-05 05:44:19.000,8b02698a-21bc-11ea-a13a-137349068a90,30048d32-7d42-11eb-8fb5-0242ac1c0002,1920,1080,8b02698a-21bc-11ea-a13a-137349068a90.jpg,0.0,0,"[[0, 0.091, 0.983, 0.876]]"
1,6,3,2013-06-05 05:44:20.000,8e5b81de-21bc-11ea-a13a-137349068a90,30048d32-7d42-11eb-8fb5-0242ac1c0002,1920,1080,8e5b81de-21bc-11ea-a13a-137349068a90.jpg,0.0,1,"[[0.193, 0.261, 0.803, 0.714]]"
2,6,3,2013-06-05 05:44:21.000,8c6be0e4-21bc-11ea-a13a-137349068a90,30048d32-7d42-11eb-8fb5-0242ac1c0002,1920,1080,8c6be0e4-21bc-11ea-a13a-137349068a90.jpg,0.0,2,"[[0.717, 0.412, 0.282, 0.558]]"
3,6,3,2013-06-05 05:44:22.000,8fdf7998-21bc-11ea-a13a-137349068a90,30048d32-7d42-11eb-8fb5-0242ac1c0002,1920,1080,8fdf7998-21bc-11ea-a13a-137349068a90.jpg,0.0,3,"[[0.772, 0.474, 0.226, 0.504]]"
4,6,3,2013-06-05 05:44:23.000,96093c50-21bc-11ea-a13a-137349068a90,30048d32-7d42-11eb-8fb5-0242ac1c0002,1920,1080,96093c50-21bc-11ea-a13a-137349068a90.jpg,0.0,4,"[[0.862, 0.539, 0.136, 0.43]]"


In [57]:
#Some files are not present at train folder
exist_list = []
for i in range(df.shape[0]):
    exist_list.append(os.path.isfile(f'./train/{df["file_name"][i]}'))
df['is_exist'] = exist_list
df.head(3)

,seq_num_frames,location,datetime,id,seq_id,width,height,file_name,sub_location,seq_frame_num,box,is_exist
0,6,3,2013-06-05 05:44:19.000,8b02698a-21bc-11ea-a13a-137349068a90,30048d32-7d42-11eb-8fb5-0242ac1c0002,1920,1080,8b02698a-21bc-11ea-a13a-137349068a90.jpg,0.0,0,"[[0, 0.091, 0.983, 0.876]]",False
1,6,3,2013-06-05 05:44:20.000,8e5b81de-21bc-11ea-a13a-137349068a90,30048d32-7d42-11eb-8fb5-0242ac1c0002,1920,1080,8e5b81de-21bc-11ea-a13a-137349068a90.jpg,0.0,1,"[[0.193, 0.261, 0.803, 0.714]]",False
2,6,3,2013-06-05 05:44:21.000,8c6be0e4-21bc-11ea-a13a-137349068a90,30048d32-7d42-11eb-8fb5-0242ac1c0002,1920,1080,8c6be0e4-21bc-11ea-a13a-137349068a90.jpg,0.0,2,"[[0.717, 0.412, 0.282, 0.558]]",False


In [58]:
df = df[df['is_exist']]
df.reset_index(drop=True,inplace=True)

In [59]:
df = df[:5]

In [60]:
df

,seq_num_frames,location,datetime,id,seq_id,width,height,file_name,sub_location,seq_frame_num,box,is_exist
0,10,10,2013-03-02 07:58:26.000,87cae3e6-21bc-11ea-a13a-137349068a90,302b4594-7d42-11eb-8fb5-0242ac1c0002,1920,1080,87cae3e6-21bc-11ea-a13a-137349068a90.jpg,NaN,7,"[[0, 0.525, 0.387, 0.446]]",True
1,10,10,2013-03-02 07:58:27.000,87763080-21bc-11ea-a13a-137349068a90,302b4594-7d42-11eb-8fb5-0242ac1c0002,1920,1080,87763080-21bc-11ea-a13a-137349068a90.jpg,NaN,8,"[[0.355, 0.579, 0.645, 0.393]]",True
2,6,336,2013-05-21 12:29:52.000,86899ed2-21bc-11ea-a13a-137349068a90,9650b9cc-21bc-11ea-a13a-137349068a90,2048,1536,86899ed2-21bc-11ea-a13a-137349068a90.jpg,NaN,2,[],True
3,3,514,2013-06-15 13:16:02.000,89220af8-21bc-11ea-a13a-137349068a90,9205274a-21bc-11ea-a13a-137349068a90,2048,1536,89220af8-21bc-11ea-a13a-137349068a90.jpg,NaN,2,[],True
4,3,229,2015-02-11 00:17:29.000,87e1aec8-21bc-11ea-a13a-137349068a90,90db6b86-21bc-11ea-a13a-137349068a90,1280,1024,87e1aec8-21bc-11ea-a13a-137349068a90.jpg,NaN,0,"[[0.228, 0.742, 0.213, 0.244]]",True


In [86]:
X,y = df['file_name'], df['box']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.reset_index(drop=True,inplace=True),X_test.reset_index(drop=True,inplace=True),y_train.reset_index(drop=True,inplace=True),y_test.reset_index(drop=True,inplace=True)

(None, None, None, None)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.labels = df['box']
        self.images = df['file_name']
        self.transform = transforms

    def __len__(self):
        return self.images.shape[0]

    def __getitem__(self, idx):
        path = 'train/' + self.images.iloc[idx]
        image = Image.open(path).convert('RGB').resize((1920, 1080))
        label = json.loads(self.labels.iloc[idx])
        if self.transform:
            image = self.transform(image)
        return image, label

BATCH_SIZE = 70
IMAGE_SIZE = 112

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


train_data = CustomDataset(df, transforms= train_transform)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

In [52]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
def get_image(path):
    path = 'train/' + path
    image = Image.open(path).convert('RGB').resize((1920, 1080))
    return train_transform(image)

# Model

In [92]:
X_images_train = [get_image(path) for path in X_train]
X_images_test = [get_image(path) for path in X_test]

boxes = [list(map(lambda s: float(s) if len(s)!=0 else 0,y[2:-2].split(', '))) for y in y_train]
boxes = torch.Tensor(boxes)
labels = torch.Tensor([1 for y in y_train])
y_train = {'boxes': boxes,'labels':labels}

boxes = [list(map(lambda s: float(s) if len(s)!=0 else 0,y[2:-2].split(', '))) for y in y_test]
boxes = torch.Tensor(boxes)
labels = torch.Tensor([1 for y in y_test])
y_test = {'boxes': boxes,'labels':labels}

ValueError: expected sequence of length 4 at dim 1 (got 1)

In [93]:
boxes

[[0.228, 0.742, 0.213, 0.244],
 [(0, 0, 0, 0)],
 [0.0, 0.525, 0.387, 0.446],
 [(0, 0, 0, 0)]]

In [ ]:
model = fasterrcnn_resnet50_fpn()
output = model(X_images_train,y_train)
model.eval()
model(X_images_test)